In [ ]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine, text
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# 1.Example of df

data = {
    'id': [1, 2, 3],
    'name': ['Alice', 'Bob', 'Charlie'],
    'score': [95, 87, 78]
}
df = pd.DataFrame(data)
df


In [ ]:
user_name=''
password=''


In [ ]:
# Conection
host = 'localhost'
port = '5433'
database = 'layereddb'
schema='berlin_source_data'

#connection to db after you opened tunnel
engine = create_engine(f'postgresql+psycopg2://{user_name}:{password}@{host}:{port}/{database}')

### Before creating the table
Before creating the table, always pay attention to the `CREATE TABLE` statement in SQL. Check what columns are included, their order, their exact names, and their data types, including the length limits. Your Pandas DataFrame must match this table exactly — same column order, same names, and compatible data types — before you send it to the database.


#### Constraints and references
Pay attention to constraints and foreign key references in your table. These are defined in the `CREATE TABLE` statement. After creating the table with constraints and references, go to DBeaver and verify that the empty table has been correctly created with all the intended constraints and references.


In [ ]:


#this is where you create table with constraints and references first
create_table_query = f"""
CREATE TABLE IF NOT EXISTS {schema}.test_table_{user_name} (
    id INTEGER PRIMARY KEY,
    name VARCHAR(100) NOT NULL,
    score INTEGER
);
"""

# Execute the query to create empty table
with engine.connect() as conn:
    conn.execute(text(create_table_query))
    conn.commit()  # commit the transaction




####  Sending the DataFrame
Once you have verified in DBeaver that the empty table exists with its references and constraints, it is time to send your Pandas DataFrame using `.to_sql()` with `if_exists='append'`. If there are mismatches in data types, column names, or order compared to the table, Python will raise an error. If everything matches, the table will be populated correctly, preserving the constraints and references.


In [ ]:
#  Send the DataFrame to the database using .to_sql()
df.to_sql(
    f'test_table_{user_name}',        # table name
    engine,
    schema=schema,
    if_exists='append', # append table if it exists
    index=False
)

print("DataFrame sent to PostgreSQL using .to_sql() with psycopg2!")

In [ ]:
##let's query test data!
query = f"""
SELECT * from berlin_source_data.districts
"""

# Execute the query
with engine.connect() as conn:
    df= pd.read_sql(text(query), conn)
    conn.commit()  # commit the transaction
df